In [1]:
from tqdm.notebook import tqdm  # 진행과정 시각화
from datetime import timedelta  # 시간날짜
import time
import re
import os
import datetime
import pandas as pd
import numpy as np
import gc

import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup  # 크롤링

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important; }</style>'))

In [3]:
def del_outword(string):
    '''크롤링 후 이모티콘, \u200b과 같은 문자가 아닌 것 제거
    [출처](https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python)
    '''
    #이모지제거
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)
    
        #분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
    han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~"^_\n\r#\ufeff\u200d\u200b\u7643\ufffd\u682e\u62c4]+')
    
    string = emoji_pattern.sub(r'',string)
    string = han.sub(r'',string)
    
    return string

def time_change(x):
    '''검색결과에 있는 문자를 날짜로 변경
    네이버 블로그 크롤링시 작성일자가 1분전/1시간전/1일전으로 나타는 경우가 가끔씩 존재
    이러한 패턴을 일반적인 YY-MM-DD형태로 변환함
    그리고 월별 계산이 중요하므로 모든 데이터는 1일로 통일
    
    Return
    '''
    min_pattern = re.compile('[0-9]+'+"분 전")
    hour_pattern = re.compile('[0-9]+'+"시간 전")
    day_pattern = re.compile('[0-9]+'+"일 전")

    today = datetime.datetime.today().date()
    # 일자
    if "일" in x :
        d = re.findall(day_pattern, x)[0][0]
        x = today - timedelta(days=int(d))
    elif "시간" in x:
        d = re.findall(hour_pattern, x)[0]
        x = today
    elif "분" in x:
        d = re.findall(min_pattern, x)[0]
        x = today
    elif x == "어제":
        x = today - timedelta(days=1)
    elif type(x) == str: # 'YY.NN.DD'형태
        x = datetime.datetime.strptime(x.replace('.', '-')[:-1], '%Y-%m-%d')
    
    #모든일자를 1일로 통일
    x = datetime.datetime.strptime(x.strftime('%Y-%m')+"-01","%Y-%m-%d")
    return x

def make_date_list(start_year,end_year,month_gap = 1,year_gap = 1, types = 'month'):
    '''검색에 사용할 날짜 list를 생성
    시작년도, 마지막년도를 입력하고 월별간격, 연별 간격을 입력하면
    검색에 사용할 "YYMMDD"형태의 날짜 list를 반환
    
    Parameters
    ----------
    start_year(int) : 검색을 시작할 연도
    end_year(int) : 검색을 종료할 연도
    month_gap(int) : 몇개월 단위로 검색할 것인지
    year_gap(int) : 몇년 단위로 검색할 것인지
    types(string) : 'year' or 'month'로 연간검색 혹은 월간검색지정
    
    Returns 
    -------
    start_date, end_date : list
        YYMMDD형태의 string형태의 날짜들은 반환한다
    
    Examples
    --------
    >>> make_date_list(start_year = 2010, end_year = 2011,month_gap = 1,year_gap = 1, types = 'month')
    start_date = ['20100101', '20100701', '20110101', '20110701']
    end_date = ['20100131', '20100731', '20110131', '20110731']
    
    >>> make_date_list(start_year = 2010, end_year = 2013,year_gap = 1, types = 'year')
    start_date = ['20100101', '20110101', '20120101', '20130101']
    end_date = ['20101231', '20111231', '20121231', '20131231']
    '''
    #일자
    
    month_gap = month_gap*100
    start_month = []
    end_month = []
    
    #MMDD 형태 추출
    for date in range(101,1202,month_gap):
        start_month.append(date)
    start_month = list(map(lambda x : '{:0>4}'.format(str(x)),start_month))

    for date in range(131,1232,month_gap):
        end_month.append(date)
    end_month = list(map(lambda x : '{:0>4}'.format(str(x)),end_month))

    start_date = []
    end_date = []
    
    #YYMMDD 형태로 추출
    if types.lower() == 'month':
        for year in range(start_year,end_year+1,year_gap):
            for date in start_month:
                start_date.append(str(year)+date)
            for date in end_month:
                end_date.append(str(year)+date)
    elif types.lower() == 'year':
        for year in range(start_year,end_year+1,year_gap):
            start_date.append(str(year)+'0101')
            end_date.append(str(year)+'1231')
            
        
    return start_date,end_date


In [4]:

keyword = '캠핑'
date_option=8
date_from='20200101'
date_to='20201231'
base_url = f'https://search.naver.com/search.naver?date_from={date_from}&date_option={date_option}&date_to={date_to}&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={keyword}&sm=tab_pge&srchby=all&st=sim&where=post'
path = 'new_output'
#저장위치
os.makedirs('./new_output',exist_ok=True)

In [8]:
#함수화 


def new_crawling(keyword, path, date_from='', date_to='', save=True):
    '''네이버 블로그 검색결과가 드래그 방식으로 바뀜(기존 페이지방식)
    따라서 셀레니움으로 최대(1000건) 드래그후 작성일자, 제목, url을 가져온 후 
    request로 각 블로그에 접속하여 본문추출
    이 과정에서 네이버블로그가 아닌 블로그 제외됨(tistory, daum등)
    
    parameter
    ---------
    keyword(string) : 검색하고 싶은 키워드 "keyword +필수어" 형태로 필수단어 추가 가능
    path(string) : 저장할 폴더 설정
    date_from(YYYYMMDD) : (default = "") date_option이 8일때 사용 검색 시작일자를 지정
    date_to(YYYYMMDD) : (default = ""), date_option이 8일때 사용 검색 마지막일자를 지정
    save(bool) : (default = True)csv로 저장 여부 결정
    
    Returns
    ---------
    df : DataFrame
      post_dates title  full_text         url
    0 2010-01-01 title  [full_text]       http://blog.naver.com/PostView.nhn?blogId=wend...  
    
    '''
    
    base_url = f'https://search.naver.com/search.naver?date_from={date_from}&date_option=8&date_to={date_to}&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={keyword}&sm=tab_pge&srchby=all&st=sim&where=post'
    
    #임시저장
    blog_urls = dict()
    blog_urls['post_dates'] = []
    blog_urls['title'] = []
    blog_urls['url'] = []
    options = Options()
    
     # 크롬드라이버 작동 
    options.add_argument('--start-maximized') #전체화면
    options.add_argument('--headless') #headless(화면안켜고)
    driver = webdriver.Chrome(executable_path='E:/Ella/RStudio/DataAnalysis/프로젝트/chromedriver.exe', chrome_options=options)
    driver.get(base_url)

    #최대 아래로 스크롤
    count = 1
    b_blog_num = ''
    try :
        for i in range(0,40):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.5)
            count += 1
#             print(count)
            #스크롤 마지막 blog_num 비교
            source = driver.page_source
            soup = BeautifulSoup(source, 'html.parser')
            l_blog_num = soup.select('li[id^="sp_blog"]')[-1]['id']

            if b_blog_num == l_blog_num: #마지막 번호와 동일하면 스크롤 중지
                break
            else :
                b_blog_num = l_blog_num

        source = driver.page_source
        # print(count)
        driver.close()

        #각 정보 불러오기
        soup = BeautifulSoup(source, 'html.parser')

        #dates
        dates_list = soup.select('div > div.total_sub > span > span > span.etc_dsc_area > span')
        dates = pd.Series(dates_list).apply(lambda x : x.text)
        blog_urls['post_dates'] = blog_urls['post_dates'] + list(dates)

        #titles
        title_list = soup.select('div > a.api_txt_lines.total_tit')
        titles = pd.Series(title_list).apply(lambda x : x.text)
        blog_urls['title'] = blog_urls['title'] + list(titles)

        #link
        link_list = soup.select('div.total_dsc_wrap a.total_dsc')
        links = pd.Series(link_list).apply(lambda x : x.get('href'))
        blog_urls['url'] = blog_urls['url'] + list(links)

        #중복제거
        df = pd.DataFrame(blog_urls)
        df = df.drop_duplicates('url')
        blog_urls['url'] = df['url']
        ori_length = len(blog_urls['url'])

        # 본문가져오기
        output_error = []
        texts = []
        for i, url in tqdm(enumerate(blog_urls['url'])):
            count = i
            try:
                # 블로그 url안에 들어가기(아직 크롤링불가)
                get_blog_post_content_code = requests.get(url)
                get_blog_post_content_text = get_blog_post_content_code.text
                get_blog_post_content_soup = BeautifulSoup(
                    get_blog_post_content_text, 'lxml')

                # 크롤링가능한 url에 접속
                real_blog_post_url = "http://blog.naver.com" + get_blog_post_content_soup.select('#mainFrame')[0].get('src')
                get_real_blog_post_content_code = requests.get(real_blog_post_url)
                get_real_blog_post_content_text = get_real_blog_post_content_code.text
                get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')


                # 본문  (postviewarea 패턴과 se-main-container 2가지 유형이 있어 분리함)
                try:
                    blog_post_content = get_real_blog_post_content_soup.select('div.se-main-container')
                    if len(blog_post_content) == 0:
                        blog_post_content = get_real_blog_post_content_soup.select('div#postViewArea')
                        if len(blog_post_content) == 0:
                            blog_post_content = get_real_blog_post_content_soup.select('div.se_component_wrap.sect_dsc.__se_component_area')

                    string = ""
                    for sentence in blog_post_content[0].stripped_strings:
                        try :
                            string += " "+sentence.replace('\xa0', " ")
                        except :
                            pass
                        # 비언어 텍스트제거
                    string = del_outword(string)
                    # 공백에러대처
                    blank_check = string.replace(" ", "")
                    if len(blank_check) == 0:
                        texts.append(["X"])
                    else:
                        texts.append([string])
                    count += 1
                except Exception as ex:
                    print(ex)
                    texts.append(["X"])
                    output_error.append(count)
                    count += 1
      

            except Exception as ex:
    #             print('가져오기에러 {num}번째'.format(num=count), ex)
                output_error.append(count)
                count += 1
                pass

        #후처리
        df = df.drop(output_error) #에러제거
        df['full_text'] = texts
        df['post_dates'] = df['post_dates'].apply(lambda x : time_change(x)) #날짜 일반화
        last_length = len(output_error)
        print(f'검색된길이:{ori_length},제외된 길이:{last_length}')
        fin_length = df.shape[0]
        #저장
        if save == True:
            today = datetime.datetime.today().date()
            str(today)
            os.makedirs(f'./{path}/크롤링/{keyword}',exist_ok=True)
            df.to_csv(f'./{path}/크롤링/{keyword}/{date_from}~{date_to}_{keyword}_{fin_length}_{today}.csv',index=False)
    except Exception as ex:
        print(ex)
        pass
    return df

 
    

In [10]:
#10년치 7시간반
start_dates, end_dates = make_date_list(2021,2021)

for date_num in tqdm(range(0, len(start_dates))):
    date_from = start_dates[date_num]
    date_to = end_dates[date_num]
    new_crawling(keyword, path, date_from, date_to,True)

    

C:\Users\hongm\anaconda3\envs\ck\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: use options instead of chrome_options



검색된길이:570,제외된 길이:8



검색된길이:420,제외된 길이:0



검색된길이:630,제외된 길이:1



검색된길이:600,제외된 길이:0


...


검색된길이:450,제외된 길이:0



검색된길이:330,제외된 길이:0



검색된길이:484,제외된 길이:0



검색된길이:600,제외된 길이:0



검색된길이:450,제외된 길이:0

